In [ ]:
import seaborn as sns
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None # supprime certains warnings
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import numpy as np


In [ ]:

def features_incomplete(X):   
    Features_incomplete = []
    for x in X.columns:
        if X[x].isnull().sum()/len(X[x])*100 > 0:
            Features_incomplete.append(x)
    return Features_incomplete

def preprocess(X):
    X.drop(columns = ["exposition"],inplace=True)
    X.drop(columns=['ghg_category', 'energy_performance_category'], inplace=True)
    X['floor'].fillna(0, inplace=True)
    X['land_size'].fillna(0, inplace=True)

    #one hot encoding for property_type
    X = pd.get_dummies(X, columns=['property_type'], drop_first=False)

    # Label encoding for city
    le = LabelEncoder()
    X['city'] = le.fit_transform(X['city'])


    # Calcul des valeurs des départements plutôt que des codes postaux (en séparant les départements à 4 chiffres des départements à 5 chiffres)
    X["departement"]=X["postal_code"]
    X["departement"][X["departement"] < 10000]=X["departement"].astype(str).str[:1].astype(int)
    X["departement"][X["departement"] >= 10000]=X["departement"].astype(str).str[:2].astype(int)

    # Tentatives de remplir les valeurs manquantes de ces colonnes par la valeur moyenne pour le code postal associé
    #X["POPULATION_x"].fillna(X.groupby("postal_code")["POPULATION_x"].transform("mean"), inplace=True)
    #X["Nb_Transac"].fillna(X.groupby("postal_code")["Nb_Transac"].transform("mean"), inplace=True)
    #X["Nb_Ventes"].fillna(X.groupby("postal_code")["Nb_Ventes"].transform("mean"), inplace=True)
    
    X["nearly_price"]=X["PrixMoyen_M2"]*X["size"]
    X["nearly_price_1819"]=X["PrixMoyen_M2_1819"]*X["size"]
    X["prix_metre"] = np.sqrt(X["PrixMoyen_M2"])
    X["prix_metre_1819"] = np.sqrt(X["PrixMoyen_M2_1819"])
    # X["prix_metre_2017"] = np.sqrt(X["Prix_2017"])
    # X["prix_metre_2018"] = np.sqrt(X["Prix_2018"])
    # X["racine_size"] = np.sqrt(X["size"])
    # X["racine_land_size"] = np.sqrt(X["land_size"])
    
    X["Somme_bedrooms_rooms"] = X["nb_rooms"]+X["nb_bedrooms"]
    X["Somme_bathrooms_rooms"] = X["nb_rooms"]+X["nb_bathrooms"]
    X["Diff_bedrooms_bathrooms"] = X["nb_bathrooms"]-X["nb_bedrooms"]

  
    
    

    # Prendre l'entier de la longitude et de la latitude
    X["approximate_longitude"]=X["approximate_longitude"].astype(int)
    X["approximate_latitude"]=X["approximate_latitude"].astype(int)
    
    
    for x in ["approximate_longitude","approximate_latitude","nearly_price"]:
    # On remplit le reste par la valeur moyenne du département
        X[x].fillna(X.groupby("departement")[x].transform("mean"), inplace=True)
    


    
    X = X.drop(columns = ["id_annonce"],axis=1)
    
    return X

In [ ]:
pipeline = make_pipeline(MinMaxScaler(), KMeans(n_clusters=15),MinMaxScaler(), xgb.XGBRegressor())